Техніка моделювання: класифікація.

Класифікація є найбільш релевантним підходом для вирішення нашої бізнес-проблеми, оскільки нам необхідно класифікувати діагноз пацієнтів та наша цільова змінна є категоріальною.


Метрики: precision, recall, f1-score

Precision – оцінка того, скільки даних передбачених як певний клас, дійсно відносяться до цього класу.

Recall – оцінка того, скільки даних певного класу ми знайшли відносно загальної кількості даних в цьому класі.

F1-score – оцінка середнього зваженого precision та recall. Оптимальний вибір для оцінки моделі, так як можна одразу оцінити наскільки хороші precision та recall у нашої моделі.

Всі ці метрики застосовуються в задачах класифікації.


In [ ]:
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine

load_dotenv()

engine = create_engine(os.getenv("DATABASE_URL"))

dataset = pd.read_csv("../data/clean_dataset.csv")

y = dataset['diagnosis']
X = dataset.drop(columns='diagnosis')

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, random_state=7, stratify=y_temp)

print(dataset.shape)
print(X_train.shape)

(981, 23)
(686, 22)


In [9]:
from sklearn.metrics import f1_score, precision_score, recall_score
from datetime import datetime

time_now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def train_model(model, model_name, X_train, y_train, X_test, y_test, metrics_data):
    model.fit(X_train, y_train)

    pred = model.predict(X_test)

    score_f1 = f1_score(y_test, pred, average='weighted')
    score_recall = recall_score(y_test, pred, average='weighted')
    score_precision = precision_score(y_test, pred, average='weighted')

    metrics_data.append(
        {
            'timestamp': [time_now],
            'model': [model_name],
            'f1_score': [score_f1],
            'recall': [score_recall],
            'precision': [score_precision],
        }
    )

    print(f'{model_name}, f1_score: {score_f1}')
    print(f'{model_name}, recall: {score_recall}')
    print(f'{model_name}, precision: {score_precision}\n')

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

model_lr = LogisticRegression()
model_rfc = RandomForestClassifier()
model_svc = SVC()

all_metrics_data = []

train_model(model_lr, 'LR', X_train, y_train, X_test, y_test, all_metrics_data)
train_model(model_rfc, 'RFC', X_train, y_train, X_test, y_test, all_metrics_data)
train_model(model_svc, 'SVC', X_train, y_train, X_test, y_test, all_metrics_data)

new_metrics_df = pd.DataFrame(all_metrics_data)
csv_file_path = 'model_metrics.csv'

if os.path.exists(csv_file_path):
    new_metrics_df.to_csv(csv_file_path, mode='a', header=False, index=False)
    print(f"Результати додано до існуючого файлу: {csv_file_path}")
else:
    new_metrics_df.to_csv(csv_file_path, mode='w', header=True, index=False)
    print(f"Створено новий файл та збережено результати: {csv_file_path}")

LR, f1_score: 0.9580372250423013
LR, recall: 0.9593908629441624
LR, precision: 0.9630826026765112

RFC, f1_score: 1.0
RFC, recall: 1.0
RFC, precision: 1.0

SVC, f1_score: 0.9119936883159414
SVC, recall: 0.9187817258883249
SVC, precision: 0.9323181049069375

Результати додано до існуючого файлу: model_metrics.csv


In [11]:
from sklearn.inspection import permutation_importance

result = permutation_importance(model_svc, X_test, y_test, n_repeats=20, random_state=7)
print('Result is calculated')

Result is calculated


In [ ]:
try:
    feature_names = X_train.columns.tolist()
except AttributeError:
    num_features = len(model_rfc.feature_importances_)
    feature_names = [f'feature_{i}' for i in range(num_features)]
    print("Не вдалося отримати назви колонок з 'dataset'. Використовуються назви за замовчуванням.\n")

pd.set_option('display.max_rows', None)

print("## Важливість ознак: Logistic Regression ##")
for i in range(model_lr.coef_.shape[0]):
    print(f"\n--- Коефіцієнти для Класу {i} ---")
    lr_df = pd.DataFrame({
        'Ознака': feature_names,
        'Коефіцієнт': abs(model_lr.coef_[i])
    })
    lr_df_sorted = lr_df.reindex(lr_df.Коефіцієнт.abs().sort_values(ascending=False).index)
    print(lr_df_sorted.to_string(index=False))

print("\n" + "="*50 + "\n")

print("## Важливість ознак: Random Forest ##")
rf_df = pd.DataFrame({
    'Ознака': feature_names,
    'Важливість': abs(model_rfc.feature_importances_)
}).sort_values(by='Важливість', ascending=False)

print(rf_df.to_string(index=False))

print("\n" + "="*50 + "\n")

print("## Важливість ознак: SVM (Permutation Importance) ##")
svm_df = pd.DataFrame({
    'Ознака': feature_names,
    'Зменшення якості': abs(result.importances_mean)
}).sort_values(by='Зменшення якості', ascending=False)

print(svm_df.to_string(index=False))

pd.reset_option('display.max_rows')

## Важливість ознак: Logistic Regression ##

--- Коефіцієнти для Класу 0 ---
                         Ознака  Коефіцієнт
          depression_phq9_score    3.149339
                    pem_present    2.056713
       exercise_frequency_Often    0.354015
            work_status_Working    0.337006
 social_activity_level_Very low    0.275314
  work_status_Partially working    0.233125
       exercise_frequency_Never    0.210902
  meditation_or_mindfulness_Yes    0.209401
      social_activity_level_Low    0.177470
      exercise_frequency_Rarely    0.155297
                    gender_Male    0.115478
   exercise_frequency_Sometimes    0.107828
                brain_fog_level    0.102994
   fatigue_severity_scale_score    0.102084
            sleep_quality_index    0.096150
   social_activity_level_Medium    0.084506
             pem_duration_hours    0.075841
                            age    0.045998
       hours_of_sleep_per_night    0.021782
            physical_pain_score    0.020844

In [15]:
dataset = pd.read_csv("../data/clean_dataset_2.csv")

y = dataset['diagnosis']
X = dataset.drop(columns='diagnosis')

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.125, random_state=7, stratify=y_temp)

print(dataset.shape)
print(X_train.shape)

(981, 21)
(686, 20)


In [16]:
model_lr = LogisticRegression()
model_rfc = RandomForestClassifier()
model_svc = SVC()

all_metrics_data = []

train_model(model_lr, 'LR', X_train, y_train, X_test, y_test, all_metrics_data)
train_model(model_rfc, 'RFC', X_train, y_train, X_test, y_test, all_metrics_data)
train_model(model_svc, 'SVC', X_train, y_train, X_test, y_test, all_metrics_data)

new_metrics_df = pd.DataFrame(all_metrics_data)
csv_file_path = 'model_metrics.csv'

if os.path.exists(csv_file_path):
    new_metrics_df.to_csv(csv_file_path, mode='a', header=False, index=False)
    print(f"Результати додано до існуючого файлу: {csv_file_path}")
else:
    new_metrics_df.to_csv(csv_file_path, mode='w', header=True, index=False)
    print(f"Створено новий файл та збережено результати: {csv_file_path}")

LR, f1_score: 0.6050043763312248
LR, recall: 0.6091370558375635
LR, precision: 0.6295607316858285

RFC, f1_score: 0.6307017777553853
RFC, recall: 0.6649746192893401
RFC, precision: 0.6510524425434518

SVC, f1_score: 0.5875671363820811
SVC, recall: 0.6345177664974619
SVC, precision: 0.6370212765957446

Результати додано до існуючого файлу: model_metrics.csv


In [18]:
result = permutation_importance(model_svc, X_test, y_test, n_repeats=20, random_state=7)
print('Result is calculated')

Result is calculated


In [19]:
try:
    feature_names = X_train.columns.tolist()
except AttributeError:
    num_features = len(model_rfc.feature_importances_)
    feature_names = [f'feature_{i}' for i in range(num_features)]
    print("Не вдалося отримати назви колонок з 'dataset'. Використовуються назви за замовчуванням.\n")

pd.set_option('display.max_rows', None)

print("## Важливість ознак: Logistic Regression ##")
for i in range(model_lr.coef_.shape[0]):
    print(f"\n--- Коефіцієнти для Класу {i} ---")
    lr_df = pd.DataFrame({
        'Ознака': feature_names,
        'Коефіцієнт': abs(model_lr.coef_[i])
    })
    lr_df_sorted = lr_df.reindex(lr_df.Коефіцієнт.abs().sort_values(ascending=False).index)
    print(lr_df_sorted.to_string(index=False))

print("\n" + "="*50 + "\n")

print("## Важливість ознак: Random Forest ##")
rf_df = pd.DataFrame({
    'Ознака': feature_names,
    'Важливість': abs(model_rfc.feature_importances_)
}).sort_values(by='Важливість', ascending=False)

print(rf_df.to_string(index=False))

print("\n" + "="*50 + "\n")

print("## Важливість ознак: SVM (Permutation Importance) ##")
svm_df = pd.DataFrame({
    'Ознака': feature_names,
    'Зменшення якості': abs(result.importances_mean)
}).sort_values(by='Зменшення якості', ascending=False)

print(svm_df.to_string(index=False))

pd.reset_option('display.max_rows')

## Важливість ознак: Logistic Regression ##

--- Коефіцієнти для Класу 0 ---
                         Ознака  Коефіцієнт
   fatigue_severity_scale_score    0.528140
       exercise_frequency_Often    0.511743
 social_activity_level_Very low    0.487761
            work_status_Working    0.316797
       exercise_frequency_Never    0.311667
   social_activity_level_Medium    0.290964
  work_status_Partially working    0.259720
  meditation_or_mindfulness_Yes    0.231695
                    gender_Male    0.219142
      exercise_frequency_Rarely    0.216396
   exercise_frequency_Sometimes    0.165888
            sleep_quality_index    0.149117
      social_activity_level_Low    0.144813
                   stress_level    0.109282
social_activity_level_Very high    0.077704
                brain_fog_level    0.033288
             pem_duration_hours    0.030486
       hours_of_sleep_per_night    0.026584
                            age    0.015525
            physical_pain_score    0.007785

In [20]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

model_rfc = RandomForestClassifier(random_state=7)

param_dist_rfc = {
    'n_estimators': randint(200, 1000),
    'max_depth': [None] + list(range(10, 111, 10)),
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 11),
    'max_features': ['sqrt', 'log2', None]
}

# scoring='f1' - вказує, що ми хочемо максимізувати F1-score
# n_iter=100 - кількість комбінацій для перевірки
# cv=5 - 5-кратна крос-валідація
# n_jobs=-1 - використовувати всі доступні ядра процесора
random_search_rfc = RandomizedSearchCV(
    estimator=model_rfc,
    param_distributions=param_dist_rfc,
    n_iter=100,
    cv=5,
    scoring='f1_weighted',
    random_state=7,
    n_jobs=-1,
    verbose=2 # Показувати прогрес
)

random_search_rfc.fit(X_train, y_train)

print(f"Найкращі гіперпараметри для Random Forest: {random_search_rfc.best_params_}")
print(f"Найкращий F1-score на крос-валідації: {random_search_rfc.best_score_:.4f}")

best_model_rfc = random_search_rfc.best_estimator_

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Найкращі гіперпараметри для Random Forest: {'max_depth': 90, 'max_features': None, 'min_samples_leaf': 5, 'min_samples_split': 4, 'n_estimators': 311}
Найкращий F1-score на крос-валідації: 0.6998


In [21]:
import optuna
from sklearn.model_selection import cross_val_score, StratifiedKFold


def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
        'max_depth': trial.suggest_int('max_depth', 10, 100),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 11),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 11),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    }

    model = RandomForestClassifier(random_state=7, **params)

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
    score = cross_val_score(model, X_train, y_train, cv=cv, scoring='f1_weighted', n_jobs=-1).mean()

    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100) 

print(f"Кількість завершених спроб: {len(study.trials)}")
print(f"Найкращі гіперпараметри: {study.best_params}")
print(f"Найкращий F1-score на крос-валідації: {study.best_value:.4f}")

best_params = study.best_params
best_model_rfc_optuna = RandomForestClassifier(random_state=7, **best_params)
best_model_rfc_optuna.fit(X_train, y_train)

[I 2025-10-31 20:11:06,621] A new study created in memory with name: no-name-470dbaa7-237f-48e5-b9a5-5be066a53511
[I 2025-10-31 20:11:07,559] Trial 0 finished with value: 0.6534019890375167 and parameters: {'n_estimators': 300, 'max_depth': 54, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 'log2'}. Best is trial 0 with value: 0.6534019890375167.
[I 2025-10-31 20:11:08,855] Trial 1 finished with value: 0.6387777450956434 and parameters: {'n_estimators': 530, 'max_depth': 29, 'min_samples_split': 8, 'min_samples_leaf': 7, 'max_features': 'log2'}. Best is trial 0 with value: 0.6534019890375167.
[I 2025-10-31 20:11:09,510] Trial 2 finished with value: 0.6360344418321902 and parameters: {'n_estimators': 265, 'max_depth': 89, 'min_samples_split': 6, 'min_samples_leaf': 10, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.6534019890375167.
[I 2025-10-31 20:11:13,016] Trial 3 finished with value: 0.6837236645533824 and parameters: {'n_estimators': 999, 'max_depth': 56, '

Кількість завершених спроб: 100
Найкращі гіперпараметри: {'n_estimators': 481, 'max_depth': 34, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_features': None}
Найкращий F1-score на крос-валідації: 0.7007


,n_estimators,481
,criterion,'gini'
,max_depth,34
,min_samples_split,4
,min_samples_leaf,4
,min_weight_fraction_leaf,0.0
,max_features,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [22]:
all_metrics_data = []

train_model(best_model_rfc, 'RFC randomize', X_train, y_train, X_test, y_test, all_metrics_data)
train_model(best_model_rfc_optuna, 'RFC optuna', X_train, y_train, X_test, y_test, all_metrics_data)

new_metrics_df = pd.DataFrame(all_metrics_data)
csv_file_path = 'model_metrics.csv'

if os.path.exists(csv_file_path):
    new_metrics_df.to_csv(csv_file_path, mode='a', header=False, index=False)
    print(f"Результати додано до існуючого файлу: {csv_file_path}")
else:
    new_metrics_df.to_csv(csv_file_path, mode='w', header=True, index=False)
    print(f"Створено новий файл та збережено результати: {csv_file_path}")

RFC randomize, f1_score: 0.6845556659533215
RFC randomize, recall: 0.7157360406091371
RFC randomize, precision: 0.6764644167462432

RFC optuna, f1_score: 0.6766050139059068
RFC optuna, recall: 0.700507614213198
RFC optuna, precision: 0.670638525247128

Результати додано до існуючого файлу: model_metrics.csv


In [44]:
import joblib

joblib.dump(best_model_rfc, '../models/best_model.joblib')
print("Success saving")

Success saving


Висновок:

    У цій лабораторній роботі я протестував такі моделі класифікації: логістична регресія, метод опорних векторів та Random Forest. Тестував на двох датасетах: перший із витоком цільової змінної, другий без. Результати моделей були вищі після тренування на датасеті із витоком цільової змінної, але такий підхід до тренування моделі на датасеті з витоком цільової змінної є не правильним, тому фінальну модель будемо обирати за результатами тестування на 2 датасеті. Найкращої моделю за тестами на 2 датасеті вийшов Random Forest: f1-score = 0.6471015706560924 без підбору кращих гіперпараметрів, після підбору гіперпараметрів точність моделі підвищилася. За Random підбором гіперпараметрів результат вийшов: f1-score = 0.6845556659533215. За Optuna методом: f1-score = 0.6802889631949738. Отож, найкращою моделю вийшла модель Random Forest з підбором гіперпараметрів за RandomSearch.